In [65]:
import pandas as pd
import nfl_data_py as nfl
import re

In [66]:
flexData = pd.read_parquet('flex_predictions.parquet')
qbData = pd.read_parquet('qb_predictions.parquet')

#### Ideal Ordering
1. Player Name
2. Position
3. Range
4. Team
5. Age
6. Last Year Rank
7. Last Year Points

In [67]:
flexData.head()

,gsis_id,name,position,team,prediction,age,fantasy_points_ppr_per_game,fantasy_rank
0,00-0032764,Derrick Henry,RB,TEN,16-18,27,24.162500,22
1,00-0036223,Jonathan Taylor,RB,IND,16-18,22,21.947059,1
2,00-0033699,Austin Ekeler,RB,LAC,14-16,26,21.487500,2
3,00-0033856,Leonard Fournette,RB,TBB,12-14,26,18.257143,6
4,00-0033280,Christian McCaffrey,RB,CAR,14-16,25,18.214286,38


In [68]:
flexData['last_season_rank'] = flexData['fantasy_points_ppr_per_game'].rank(ascending=False)
flexData.rename(columns={'fantasy_points_ppr_per_game': 'last_season_points_per_game', 'prediction': 'predicted_range', 'gsis_id': 'id'}, inplace = True)
flexData = flexData[['id', 'name', 'position', 'predicted_range', 'team', 'age', 'last_season_rank', 'last_season_points_per_game']].copy()

In [69]:
flexData.head()

,id,name,position,predicted_range,team,age,last_season_rank,last_season_points_per_game
0,00-0032764,Derrick Henry,RB,16-18,TEN,27,2.0,24.162500
1,00-0036223,Jonathan Taylor,RB,16-18,IND,22,3.0,21.947059
2,00-0033699,Austin Ekeler,RB,14-16,LAC,26,5.0,21.487500
3,00-0033856,Leonard Fournette,RB,12-14,TBB,26,9.0,18.257143
4,00-0033280,Christian McCaffrey,RB,14-16,CAR,25,10.0,18.214286


In [70]:
qbData.head()

,gsis_id,name,team,position,range,age,fantasy_points_per_game
0,00-0023459,Aaron Rodgers,GBP,QB,18-21,38.0,20.768750
1,00-0019596,Tom Brady,TBB,QB,15-18,44.0,22.043529
2,00-0036355,Justin Herbert,LAC,QB,18-21,23.0,22.397647
3,00-0026498,Matthew Stafford,LAR,QB,15-18,33.0,19.396471
4,00-0034857,Josh Allen,BUF,QB,21-24,25.0,23.681176


In [71]:
qbData['last_season_rank'] = qbData['fantasy_points_per_game'].rank(ascending=False)
qbData.rename(columns={'fantasy_points_per_game': 'last_season_points_per_game', 'range': 'predicted_range', 'gsis_id': 'id'}, inplace = True)
qbData = qbData[['id', 'name', 'position', 'predicted_range', 'team', 'age', 'last_season_rank', 'last_season_points_per_game']].copy()

In [72]:
qbData.head()

,id,name,position,predicted_range,team,age,last_season_rank,last_season_points_per_game
0,00-0023459,Aaron Rodgers,QB,18-21,GBP,38.0,7.0,20.768750
1,00-0019596,Tom Brady,QB,15-18,TBB,44.0,3.0,22.043529
2,00-0036355,Justin Herbert,QB,18-21,LAC,23.0,2.0,22.397647
3,00-0026498,Matthew Stafford,QB,15-18,LAR,33.0,11.0,19.396471
4,00-0034857,Josh Allen,QB,21-24,BUF,25.0,1.0,23.681176


In [73]:
completeData = pd.concat([flexData, qbData], axis=0)

In [74]:
completeData.head()

,id,name,position,predicted_range,team,age,last_season_rank,last_season_points_per_game
0,00-0032764,Derrick Henry,RB,16-18,TEN,27.0,2.0,24.162500
1,00-0036223,Jonathan Taylor,RB,16-18,IND,22.0,3.0,21.947059
2,00-0033699,Austin Ekeler,RB,14-16,LAC,26.0,5.0,21.487500
3,00-0033856,Leonard Fournette,RB,12-14,TBB,26.0,9.0,18.257143
4,00-0033280,Christian McCaffrey,RB,14-16,CAR,25.0,10.0,18.214286


In [75]:
keys = []
for row in completeData.itertuples():
    predictedRange = row.predicted_range
    # If less than
    lessThanResults = re.search(r'<\d+', predictedRange)
    if lessThanResults:
        keys.append(0)
    else:
        greaterThanResults = re.search(r'\d+<', predictedRange)
        if greaterThanResults:
            keys.append(float('inf'))
        else:
            rangeResults = re.search(r'\d+-', predictedRange)
            if rangeResults:
                keys.append(int(predictedRange[rangeResults.start(): rangeResults.end() - 1]))
            else:
                keys.append(0)
    

In [76]:
completeData['key'] = keys
completeData.head()

,id,name,position,predicted_range,team,age,last_season_rank,last_season_points_per_game,key
0,00-0032764,Derrick Henry,RB,16-18,TEN,27.0,2.0,24.162500,16
1,00-0036223,Jonathan Taylor,RB,16-18,IND,22.0,3.0,21.947059,16
2,00-0033699,Austin Ekeler,RB,14-16,LAC,26.0,5.0,21.487500,14
3,00-0033856,Leonard Fournette,RB,12-14,TBB,26.0,9.0,18.257143,12
4,00-0033280,Christian McCaffrey,RB,14-16,CAR,25.0,10.0,18.214286,14


In [77]:
completeData.to_parquet('prodPredictionData.parquet')